## Job Shop Scheduling



<p style="text-indent: 1.5em">다품종 소량생산하는 제품들의 일정계획, 다양한 제품군에 대응해야 하므로, CNC와 같은 범용 기계 사용, 여러 제품군이 제품 별 상이한 공정계획, Setup 등에 기반하여 운용되기 때문에 생산 효율 제고를 위해서는 정밀한 생산계획 요구</p>

<p style="text-indent: 1.5em">Job, Operation, Machine 간 관계를 설정하고 개별 Job들을 수행할 수 있도록 구성하는 Operations을 Machine에 할당하는 것입니다.</p>

<p style="text-indent: 1.5em"></p>

<p style="text-indent: 1.5em"></p>

<table>
    <caption></caption>
    <tr>
        <th colspan="1"><b>Metrics</b></th>
        <th><b>Description</b></th>
    </tr>
    <tr> 
        <td>Makespan</td>
        <td align="left">할당된 작업이 모두 끝나는 시점</td>
    </tr>
    <tr> 
        <td>Mean completion time</td>
        <td>각 작업이 끝나는 시점의 평균</td>
    </tr>
    <tr> 
        <td>Maximum flow time</td>
        <td>작업이 작업에서 작업하는 시간을 합산하였을 때 가장 큰 값</td>
    </tr>
    <tr> 
        <td>Mean flow time</td>
        <td>작업이 작업장에서 작업하는 시간을 합하여 평균낸 값</td>
    </tr>
    <tr> 
        <td>Total tardiness</td>
        <td>작업의 납기시간과 완료시간의 차를 합한 값</td>
    </tr>
    <tr> 
        <td>Average tardiness</td>
        <td></td>
    </tr>
    <tr> 
        <td>Total weighted tardiness</td>
        <td></td>
    </tr>
    <tr> 
        <td>Maximum lateness</td>
        <td></td>
    </tr>
    <tr> 
        <td>Number of tardy jobs</td>
        <td></td>
    </tr>
    <tr> 
        <td>Total workload of machines</td>
        <td></td>
    </tr>
    <tr> 
        <td>Critical machine workload</td>
        <td></td>
    </tr>
</table>


**Pinedo, M. L. (2012). _Scheduling Theory, Algorithms, and Systems_. Springer, 186.**

<p style="text-indent: 1.5em">Pinedo 교수님의 책에서 예제를 발췌하였습니다. 아래는 <b>Example 7.1.1 (Disjunctive Programming Formulation)</b> 문제입니다. 아래의 표는 주어진 문제의 입력값으로 작업, 기계 할당 순서, 각 기계에서 걸리는 작업시간입니다.</p>

<table>
    <caption></caption>
    <tr>
        <th colspan="1"><b>jobs</b></th>
        <th><b>machine sequence</b></th>
        <th><b>processing times</b></th>
    </tr>
    <tr> 
        <td align="center">1</td>
        <td align="center">1, 2, 3</td>
        <td align="center">$p_{11}=10$, $p_{21}=8$, $p_{31}=4$,</td>
    </tr>
    <tr> 
        <td align="center">2</td>
        <td align="center">2, 1, 4, 3</td>
        <td align="center">$p_{22}=8$, $p_{12}=3$, $p_{42}=5$, $p_{32}=6$</td>
    </tr>
    <tr> 
        <td align="center">3</td>
        <td align="center">1, 2, 4</td>
        <td align="center">$p_{13}=4$, $p_{23}=7$, $p_{43}=3$</td>
    </tr>
</table>

<p style="text-indent: 1.5em">자세한 내용은 책을 보시는 것을 추천하며 우리는 이 문제를 파이썬 PulP를 사용하여 최적해를 찾는 것을 알아보겠습니다.</p>

$$\begin{align*}
  & \text{minimize }   &   C_{\max}   &                   &   &   &\quad(1.1)\\[1ex]
  & \text{subject to } & \,    y_{kj}-& y_{ij} \ge p_{ij} & \forall (i,j) & \rightarrow (k,j) \in A  &\quad(1.2)\\[1ex]
  &                    & \,  C_{\max}-& y_{ij} \ge p_{ij} & \forall (i,j) & \in N  &\quad(1.3)\\[1ex]  
  &                    & \,    y_{ij}-& y_{il} \ge p_{il} \quad\text{or}\quad y_{il}-y_{ij} \ge p_{ij} & \forall (i,l) & \:\text{and} \:(i,j),\quad i=1,...,m  &\quad(1.4)\\[1ex] 
  &                    & \,    & y_{ij} \ge 0             & \forall (i,j) & \in N  &\quad(1.5)\\
\end{align*}$$

In [3]:
import numpy as np
from pulp import *


n_machines = 4
n_jobs = 3
M = 10000

seq = [
    [1, 2, 3], 
    [2, 1, 4, 3], 
    [1, 2, 4]
]

pt = [
    [10, 8, 4], 
    [8, 3, 5, 6], 
    [4, 7, 3]
]

$i=\text{기계}(i=1,2,3,...,M)\\
j=작업(\:j=1,2,3,...,J)\\
x_{ij}=\text{작업}\: j\text{에서 기계}\: i\text{에서의 시작시간}\\
y_{ijl}=\text{기계}\: i\text{에서 작업}\: j\text{가 작업}\:l\text{보다 선행하면, 1}$

In [3]:
x_list = [(i, j+1) for j, _ in enumerate(machines) for i in _]
x = LpVariable.dicts(name='x', indexs=x_list, lowBound=0, cat='Integer')
x

{(1, 1): x_(1,_1),
 (2, 1): x_(2,_1),
 (3, 1): x_(3,_1),
 (2, 2): x_(2,_2),
 (1, 2): x_(1,_2),
 (4, 2): x_(4,_2),
 (3, 2): x_(3,_2),
 (1, 3): x_(1,_3),
 (2, 3): x_(2,_3),
 (4, 3): x_(4,_3)}

In [4]:
from itertools import combinations

m = 0
jj = []
jk = []
for i in range(1, num_of_machines+1):
    for _, j in enumerate(machines):
        for idx, k in enumerate(j):
            if i == k:
                jj.append(_+1)
                m += 1
    if len(jj) == m:
        jk.append(jj)
    m = 0
    jj = []
    
l = []
for j in jk:
    l.extend(combination(j, 2))
    
m = 0
idxx = []
for i, job in enumerate(jk):
    for ij, jj in enumerate(job):
        if len(job) % 2 != 0:
            idxx.append((i+1, l[m][0], l[m][1]))
            m += 1
        elif len(job) % 2 == 0:
            if ij == 0:
                idxx.append((i+1, l[m][0], l[m][1]))
                m += 1
idxx

[(1, 1, 2),
 (1, 1, 3),
 (1, 2, 3),
 (2, 1, 2),
 (2, 1, 3),
 (2, 2, 3),
 (3, 1, 2),
 (4, 2, 3)]

In [5]:
y = LpVariable.dicts(name='y', indexs=idxx, lowBound=0, upBound=1, cat='Binary')
y

{(1, 1, 2): y_(1,_1,_2),
 (1, 1, 3): y_(1,_1,_3),
 (1, 2, 3): y_(1,_2,_3),
 (2, 1, 2): y_(2,_1,_2),
 (2, 1, 3): y_(2,_1,_3),
 (2, 2, 3): y_(2,_2,_3),
 (3, 1, 2): y_(3,_1,_2),
 (4, 2, 3): y_(4,_2,_3)}

In [6]:
Cmax = LpVariable('Cmax', lowBound=0)

In [7]:
prob = LpProblem('Job Shop Scheduling', LpMinimize)

In [8]:
proc_dict = {}
for i, _ in enumerate(machines):
    for j, _ in enumerate(_):
        proc_dict[_, i+1] = processing_times[i][j]

$C_{\max}-x_{ij} \ge p_{ij}$

In [9]:
for i in x_list:
    prob += Cmax - x[i] >= proc_dict[i]

In [10]:
# for i, p in enumerate(processing_times):
#     for j, p_i in enumerate(p):
#         prob += Cmax - x[machines[i][j], i+1] >= processing_times[i][j]

$x_{kj} - x_{ij}\ge p_{ij}$

In [11]:
for i, p in enumerate(processing_times):
    for j, p_i in enumerate(p):
        if j+1 != len(p):
            prob += x[machines[i][j+1], i+1] - x[machines[i][j], i+1] >= processing_times[i][j]

In [12]:
# for i, xy in enumerate(x_list):
#     if i+1 != len(x_list):
#         prob += x[x_list[i+1]] - x[xy] >= proc_dict[xy]

In [13]:
# for i, p in enumerate(processing_times):
#     for j, p_i in enumerate(p):
#         if j+1 != len(p):
#             print(x[machines[i][j+1], i+1] - x[machines[i][j], i+1], processing_times[i][j])

$y_{ijl} - y_{ilj} \ge p_{il} - M\cdot (1-y_{ilj})$

In [14]:
for i in idxx:
    print(x[i[0], i[1]], x[i[0], i[2]], proc_dict[i[0], i[2]], y[i])

x_(1,_1) x_(1,_2) 3 y_(1,_1,_2)
x_(1,_1) x_(1,_3) 4 y_(1,_1,_3)
x_(1,_2) x_(1,_3) 4 y_(1,_2,_3)
x_(2,_1) x_(2,_2) 8 y_(2,_1,_2)
x_(2,_1) x_(2,_3) 7 y_(2,_1,_3)
x_(2,_2) x_(2,_3) 7 y_(2,_2,_3)
x_(3,_1) x_(3,_2) 6 y_(3,_1,_2)
x_(4,_2) x_(4,_3) 3 y_(4,_2,_3)


In [15]:
for i in idxx:
    prob += x[i[0], i[1]] - x[i[0], i[2]] >= proc_dict[i[0], i[2]] - M*(1-y[i])

$y_{ilj} - y_{ijl} \ge p_{il} - M\cdot y_{ilj}$

In [16]:
for i in idxx:
    print(x[i[0], i[2]], x[i[0], i[1]], proc_dict[i[0], i[1]], y[i])

x_(1,_2) x_(1,_1) 10 y_(1,_1,_2)
x_(1,_3) x_(1,_1) 10 y_(1,_1,_3)
x_(1,_3) x_(1,_2) 3 y_(1,_2,_3)
x_(2,_2) x_(2,_1) 8 y_(2,_1,_2)
x_(2,_3) x_(2,_1) 8 y_(2,_1,_3)
x_(2,_3) x_(2,_2) 8 y_(2,_2,_3)
x_(3,_2) x_(3,_1) 4 y_(3,_1,_2)
x_(4,_3) x_(4,_2) 5 y_(4,_2,_3)


In [17]:
for i in idxx:
    prob += x[i[0], i[2]] - x[i[0], i[1]] >= proc_dict[i[0], i[1]] - M*y[i]

In [18]:
prob.solve()

1

In [19]:
print('Status: ', LpStatus[prob.status])

for v in prob.variables():
    print(v.name, '=', v.varValue)

Status:  Optimal
Cmax = 10029.0
__dummy = None
x_(1,_1) = 0.0
x_(1,_2) = 26.0
x_(1,_3) = 29.0
x_(2,_1) = 10.0
x_(2,_2) = 18.0
x_(2,_3) = 33.0
x_(3,_1) = 18.0
x_(3,_2) = 34.0
x_(4,_2) = 29.0
x_(4,_3) = 10026.0
y_(1,_1,_2) = 0.0
y_(1,_1,_3) = 0.0
y_(1,_2,_3) = 0.0
y_(2,_1,_2) = 0.0
y_(2,_1,_3) = 0.0
y_(2,_2,_3) = 0.0
y_(3,_1,_2) = 0.0
y_(4,_2,_3) = 0.0


In [20]:
prob

Job Shop Scheduling:
MINIMIZE
None
SUBJECT TO
_C1: Cmax - x_(1,_1) >= 10

_C2: Cmax - x_(2,_1) >= 8

_C3: Cmax - x_(3,_1) >= 4

_C4: Cmax - x_(2,_2) >= 8

_C5: Cmax - x_(1,_2) >= 3

_C6: Cmax - x_(4,_2) >= 5

_C7: Cmax - x_(3,_2) >= 6

_C8: Cmax - x_(1,_3) >= 4

_C9: Cmax - x_(2,_3) >= 7

_C10: Cmax - x_(4,_3) >= 3

_C11: - x_(1,_1) + x_(2,_1) >= 10

_C12: - x_(2,_1) + x_(3,_1) >= 8

_C13: x_(1,_2) - x_(2,_2) >= 8

_C14: - x_(1,_2) + x_(4,_2) >= 3

_C15: x_(3,_2) - x_(4,_2) >= 5

_C16: - x_(1,_3) + x_(2,_3) >= 4

_C17: - x_(2,_3) + x_(4,_3) >= 7

_C18: x_(1,_1) - x_(1,_2) - 10000 y_(1,_1,_2) >= -9997

_C19: x_(1,_1) - x_(1,_3) - 10000 y_(1,_1,_3) >= -9996

_C20: x_(1,_2) - x_(1,_3) - 10000 y_(1,_2,_3) >= -9996

_C21: x_(2,_1) - x_(2,_2) - 10000 y_(2,_1,_2) >= -9992

_C22: x_(2,_1) - x_(2,_3) - 10000 y_(2,_1,_3) >= -9993

_C23: x_(2,_2) - x_(2,_3) - 10000 y_(2,_2,_3) >= -9993

_C24: x_(3,_1) - x_(3,_2) - 10000 y_(3,_1,_2) >= -9994

_C25: x_(4,_2) - x_(4,_3) - 10000 y_(4,_2,_3) >= -9997


## REFERENCES

1. Pinedo ()